In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings

warnings.filterwarnings(action="ignore")

import os
import tqdm
import numpy as np
import pandas as pd

from typing import Tuple, List, Dict, Any
from constants import *
from features.user import User


# 90 of the test segments are genuine (i.e., benign) and 10 segments are entered bya masquerader (randomly sorted).

In [3]:
#   load answer for classifcation
gt_df = pd.read_csv("challengeToFill.csv", index_col=0).T[TRAIN_SEGMENT_COUNT:].reset_index(drop=True)




# dev_set_df = gt_df.T.iloc[TRAIN_SEGMENT_COUNT:, :DEV_USERS_COUNT]

In [4]:
def evaluate_user_results(pred_series: pd.Series, true_series: pd.Series):
    ''' evaluate the classification result of a single user '''
    assert len(pred_series) == len(true_series)
    assert pred_series.name == true_series.name

    equal_series = pred_series == true_series
    return equal_series.mean()

In [25]:
user_list = [f"User{i}" for i in range(10)]
submission_user_list = [f"User{i}" for i in range(10, 40)]
data_path = [f"data/{uid}" for uid in user_list]
submission_data_path = [f"data/{uid}" for uid in submission_user_list]

user_data = {}

for user_id, user_data_path in tqdm.tqdm(zip(user_list, data_path), total=len(user_list)):
    #   load user data
    user_data[user_id] = User(user_id, user_data_path, gt_df[user_id])

    # create segment features
    user_data[user_id].build_segment_features()


100%|██████████| 10/10 [00:27<00:00,  2.76s/it]


In [6]:
import matplotlib.pyplot as plt

all_test_segments = [user_data[user_id].segment_df_test.select_dtypes("number") for user_id in user_list]
all_test_segments_gt = [user_data[user_id].user_anomaly_gt for user_id in user_list]

all_test_segments = pd.concat(all_test_segments).reset_index(drop=True)
all_test_segments["anomaly"] = pd.concat(all_test_segments_gt).reset_index(drop=True)


corr = all_test_segments.corr()

In [7]:
all_test_segments[all_test_segments["anomaly"] == 1].to_csv("anomalies.csv")

In [8]:
corr

,unique_cmds,longest_same_cmd_sequence,starts_with_dotfile_cmds,single_chars_cmds,two_chars_cmds,three_chars_cmds,four_chars_cmds,ends_with_dot_cmds,has_dot_in_middle_cmds,has_zip_cmds,...,not_in_train_has_all_lowercase_cmds,not_in_train_has_kill_cmds,not_in_train_has_navigation_cmds,not_in_train_has_permission_change_cmds,not_in_train_has_search_cmds,not_in_train_has_download_cmds,not_in_train_has_encryption_cmds,not_in_train_has_cmds_longer_then_8_cmds,not_in_train_has_text_documents,anomaly
unique_cmds,1.000000,-0.222672,0.022948,0.413126,0.803381,0.605625,0.605625,NaN,0.017609,0.012080,...,0.205353,0.012639,0.094474,0.005276,NaN,0.099436,NaN,NaN,0.087155,0.119453
longest_same_cmd_sequence,-0.222672,1.000000,-0.166376,0.007597,-0.215548,-0.249137,-0.249137,NaN,0.031985,0.096344,...,-0.038897,-0.008617,0.004505,-0.000536,NaN,-0.002749,NaN,NaN,-0.012271,-0.099179
starts_with_dotfile_cmds,0.022948,-0.166376,1.000000,-0.078272,-0.141681,0.019067,0.019067,NaN,0.502096,-0.006576,...,0.019442,-0.009074,0.062181,-0.009074,NaN,-0.025755,NaN,NaN,-0.015732,-0.045283
single_chars_cmds,0.413126,0.007597,-0.078272,1.000000,0.296298,0.116747,0.116747,NaN,-0.087854,-0.099746,...,-0.050330,-0.011287,0.011712,-0.011287,NaN,0.015637,NaN,NaN,-0.019569,-0.055210
two_chars_cmds,0.803381,-0.215548,-0.141681,0.296298,1.000000,0.559791,0.559791,NaN,-0.149182,-0.009649,...,0.134606,-0.026856,0.070137,0.036814,NaN,0.109010,NaN,NaN,0.100624,0.080631
three_chars_cmds,0.605625,-0.249137,0.019067,0.116747,0.559791,1.000000,1.000000,NaN,0.012823,0.052749,...,0.263480,0.014217,0.065725,0.029309,NaN,0.077836,NaN,NaN,0.050816,0.049610
four_chars_cmds,0.605625,-0.249137,0.019067,0.116747,0.559791,1.000000,1.000000,NaN,0.012823,0.052749,...,0.263480,0.014217,0.065725,0.029309,NaN,0.077836,NaN,NaN,0.050816,0.049610
ends_with_dot_cmds,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
has_dot_in_middle_cmds,0.017609,0.031985,0.502096,-0.087854,-0.149182,0.012823,0.012823,NaN,1.000000,0.083596,...,0.153595,-0.013454,0.113847,-0.013454,NaN,0.010029,NaN,NaN,-0.023327,0.144612
has_zip_cmds,0.012080,0.096344,-0.006576,-0.099746,-0.009649,0.052749,0.052749,NaN,0.083596,1.000000,...,-0.023457,-0.008846,0.022027,-0.008846,NaN,0.011387,NaN,NaN,0.044110,-0.060689


In [9]:
corr.columns

Index(['unique_cmds', 'longest_same_cmd_sequence', 'starts_with_dotfile_cmds',
       'single_chars_cmds', 'two_chars_cmds', 'three_chars_cmds',
       'four_chars_cmds', 'ends_with_dot_cmds', 'has_dot_in_middle_cmds',
       'has_zip_cmds', 'has_mail_cmds', 'has_coding_cmds', 'has_db_cmds',
       'has_help_cmds', 'has_ssh_cmds', 'has_fx_cmds', 'has_numerics_cmds',
       'has_uppercase_cmds', 'has_all_lowercase_cmds', 'has_kill_cmds',
       'has_navigation_cmds', 'has_permission_change_cmds', 'has_search_cmds',
       'has_download_cmds', 'has_encryption_cmds',
       'has_cmds_longer_then_8_cmds', 'has_text_documents',
       'cmds_not_in_train', 'not_in_train_starts_with_dotfile_cmds',
       'not_in_train_single_chars_cmds', 'not_in_train_two_chars_cmds',
       'not_in_train_three_chars_cmds', 'not_in_train_four_chars_cmds',
       'not_in_train_ends_with_dot_cmds',
       'not_in_train_has_dot_in_middle_cmds', 'not_in_train_has_zip_cmds',
       'not_in_train_has_mail_cmds', 'n

In [10]:
# import plotly.express as px
# fig = px.imshow(corr)
# fig.show()

In [17]:
''' first outlier detection algo '''
import itertools
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
from sklearn.ensemble import IsolationForest
from sklearn.metrics import precision_score, recall_score
from sklearn.decomposition import PCA
from eval.metrics import detection_metrics
from detection.pipeline import DetectionPipeline




user_scores_high_dim = {}
user_scores_low_dim = {}
session_classification = {}

for user_id in tqdm.tqdm(user_list):

    anomaly_ground_truth = user_data[user_id].user_anomaly_gt.values.astype(int) # test set only
    anomaly_ground_truth = np.array([1 if x == 0 else -1 for x in anomaly_ground_truth])

    ''' high dim data '''

    detector = DetectionPipeline()

    detector.fit(user_data[user_id].segment_df_train)
    predicted_anomalies = detector.predict(user_data[user_id].segment_df_test)
    user_scores_high_dim[user_id] = detection_metrics(predicted_anomalies, anomaly_ground_truth)

    session_classification[user_id] = [1 if x == -1 else 0 for x in predicted_anomalies]



''' aggregate metrics '''


overall_scores_high_dim = {}
for score_type in ["detection_score" ,"precision_score", "recall_score"]:
    overall_scores_high_dim[score_type] = np.mean([user_scores_high_dim[user_id][score_type] for user_id in user_list])


100%|██████████| 10/10 [00:00<00:00, 16.72it/s]


0     0
1     0
2     0
3     0
4     1
     ..
95    0
96    0
97    0
98    0
99    0
Name: User0, Length: 100, dtype: int64

In [31]:
''' submission data '''

submission_user_data = {}
session_classification = {}

for user_id, user_data_path in tqdm.tqdm(zip(submission_user_list, submission_data_path), total=len(submission_user_list)):
    #   load user data
    submission_user_data[user_id] = User(user_id, user_data_path, gt_df[user_id])

    # create segment features
    submission_user_data[user_id].build_segment_features()

    anomaly_ground_truth = submission_user_data[user_id].user_anomaly_gt.values.astype(int) # test set only
    anomaly_ground_truth = np.array([1 if x == 0 else -1 for x in anomaly_ground_truth])

    ''' high dim data '''

    detector = DetectionPipeline()

    detector.fit(submission_user_data[user_id].segment_df_train)
    predicted_anomalies = detector.predict(submission_user_data[user_id].segment_df_test)
    user_scores_high_dim[user_id] = detection_metrics(predicted_anomalies, anomaly_ground_truth)

    session_classification[user_id] = [1 if x == -1 else 0 for x in predicted_anomalies]

100%|██████████| 30/30 [01:32<00:00,  3.09s/it]


In [45]:
''' submission file '''
submission_df = pd.read_csv("challengeToFill.csv", index_col=0).T
for user_id in submission_user_list:
    submission_df[user_id].iloc[50:] = session_classification[user_id]

submission_df = submission_df.T.astype(int)
submission_df.to_csv("submissions/203763339_1.csv")

,0-100,100-200,200-300,300-400,400-500,500-600,600-700,700-800,800-900,900-1000,...,14000-14100,14100-14200,14200-14300,14300-14400,14400-14500,14500-14600,14600-14700,14700-14800,14800-14900,14900-15000
User0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
User1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
User2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
User3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
User4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
User5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
User6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
User7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
User8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
User9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
overall_scores_high_dim
# current best
# {
# 'detection_score': 0.66,
#
#  'precision_score': 0.7909999999999999,
#  'recall_score': 0.9564}

{'detection_score': 0.66,
 'precision_score': 0.7909999999999999,
 'recall_score': 0.9564}

In [13]:
overall_scores_low_dim

NameError: name 'overall_scores_low_dim' is not defined

In [ ]:
[user_scores_low_dim[user_id]["precision_score"] for user_id in user_list]

In [ ]:

import plotly.express as px

viz_df_train = pd.DataFrame({
    "x": S_train_low_dim[:, 0],
    "y": S_train_low_dim[:, 1],
    "user": TRAIN_SEGMENT_COUNT * [user_id],
    "set": TRAIN_SEGMENT_COUNT * ["train"]
})

viz_df_test = pd.DataFrame({
    "x": S_dev_low_dim[:, 0],
    "y": S_dev_low_dim[:, 1],
    "user": TEST_SEGMENT_COUNT * [user_id],
    "set": TEST_SEGMENT_COUNT * ["test"]
})


viz_df = pd.concat([viz_df_train, viz_df_test], axis=0)
viz_df["anomaly"] = gt_df.T[user_id].values
viz_df["anomaly"] = viz_df["anomaly"].astype(int)


In [ ]:
# import plotly.io as pio
# pio.renderers.default='notebook'
#
px.scatter(viz_df, x="x", y="y", color="anomaly")